#### 音素情報を用いて候補単語についてフィルタリングする

- 短い音素の単語
- 似ている音素がある単語

In [1]:
cd ../data/external/

/home/saho/project/wovle/data/external


In [92]:
!wc -l ./hybrid.htkdic

322238 ./hybrid.htkdic


In [93]:
!head ./hybrid.htkdic

<s>	[]	sp_S
</s>	[。]	sp_S
、+補助記号	[、]	sp_S
×+名詞	[×]	b_B a_I ts_I u_E
Δ+記号	[Δ]	d_B e_I r_I u_I t_I a_E
Θ+記号	[Θ]	sh_B i:_I t_I a_E
Σ+記号	[Σ]	sh_B i_I g_I u_I m_I a_E
Φ+記号	[Φ]	f_B a_I i_E
Ψ+記号	[Ψ]	p_B u_I s_I a_I i_E
Ω+名詞	[Ω]	o:_B m_I u_E


In [94]:
with open('./hybrid.htkdic') as fh:
    dic = [['\t'.join(line.split('\t')[:-1]), line.split('\t')[-1][:-1]] for line in fh]

In [95]:
dic = pd.DataFrame(dic)

In [96]:
dic.columns = ['単語','音素列']

In [97]:
dic.head()

,単語,音素列
0,<s>\t[],sp_S
1,</s>\t[。],sp_S
2,、+補助記号\t[、],sp_S
3,×+名詞\t[×],b_B a_I ts_I u_E
4,Δ+記号\t[Δ],d_B e_I r_I u_I t_I a_E


In [98]:
dic.shape

(322238, 2)

In [99]:
dic['音素'] = dic['音素列'].apply(lambda x: [xx.replace('_B','').replace('_I','').replace('_E','').replace('_S','') for xx in x.split(' ')])

In [100]:
dic['isMistaking'] = False

In [101]:
dic.head()

,単語,音素列,音素,isMistaking
0,<s>\t[],sp_S,[sp],False
1,</s>\t[。],sp_S,[sp],False
2,、+補助記号\t[、],sp_S,[sp],False
3,×+名詞\t[×],b_B a_I ts_I u_E,"[b, a, ts, u]",False
4,Δ+記号\t[Δ],d_B e_I r_I u_I t_I a_E,"[d, e, r, u, t, a]",False


### 3音素以下の単語にチェック

In [102]:
dic[('isMistaking')] = dic[('音素')].apply(lambda v: len(v) <= 3)

In [103]:
sum(dic['isMistaking'] == True)

8943

#### 音素列挙

In [104]:
dic.head()

,単語,音素列,音素,isMistaking
0,<s>\t[],sp_S,[sp],True
1,</s>\t[。],sp_S,[sp],True
2,、+補助記号\t[、],sp_S,[sp],True
3,×+名詞\t[×],b_B a_I ts_I u_E,"[b, a, ts, u]",False
4,Δ+記号\t[Δ],d_B e_I r_I u_I t_I a_E,"[d, e, r, u, t, a]",False


In [105]:
onso_list = set()
for v in dic['音素'].values:
    for w in v:
        onso_list.add(w)

In [106]:
onso_list

{'N',
 'a',
 'a:',
 'b',
 'by',
 'ch',
 'd',
 'e',
 'e:',
 'f',
 'g',
 'gy',
 'h',
 'hy',
 'i',
 'i:',
 'j',
 'k',
 'ky',
 'm',
 'my',
 'n',
 'ny',
 'o',
 'o:',
 'p',
 'py',
 'q',
 'r',
 'ry',
 's',
 'sh',
 'sp',
 't',
 'ts',
 'u',
 'u:',
 'w',
 'y',
 'z'}

### 似ている音素で一致している組み合わせがあればチェック (なんか意味無さそうなので却下)

- \* と \*:
- s と sh
- p と py

In [107]:
def isSame(a, b):
    if a == b:
        True
    elif a+':' == b or a == b+':':
        True
    elif a == 's' and b == 'sh':
        True
    elif a == 'sh' and b == 's':
        True
    elif a == 'p' and b == 'py':
        True
    elif a == 'py' and b == 'p':
        True
    else:
        False

### フラッグ付きで保存

In [108]:
dic.head()

,単語,音素列,音素,isMistaking
0,<s>\t[],sp_S,[sp],True
1,</s>\t[。],sp_S,[sp],True
2,、+補助記号\t[、],sp_S,[sp],True
3,×+名詞\t[×],b_B a_I ts_I u_E,"[b, a, ts, u]",False
4,Δ+記号\t[Δ],d_B e_I r_I u_I t_I a_E,"[d, e, r, u, t, a]",False


In [109]:
dic = dic.loc[:,['isMistaking','単語','音素列']]

In [110]:
dic['isMistaking'] = dic['isMistaking'].apply(lambda x: '1' if x else '0')

In [122]:
dic.iloc[0,:][0] = '0'
dic.iloc[1,:][0] = '0'
dic.iloc[2,:][0] = '0'

In [123]:
dic.head()

,isMistaking,単語,音素列
0,0,<s>\t[],sp_S
1,0,</s>\t[。],sp_S
2,0,、+補助記号\t[、],sp_S
3,0,×+名詞\t[×],b_B a_I ts_I u_E
4,0,Δ+記号\t[Δ],d_B e_I r_I u_I t_I a_E


In [128]:
with open('./hybrid_onso.htkdic','w') as fh:
    for d in monitor(dic.iterrows()):
        fh.write('\t'.join(d[1])+'\n')

322238it [00:36, 8829.24it/s]


In [129]:
!head ./hybrid_onso.htkdic

0	<s>	[]	sp_S
0	</s>	[。]	sp_S
0	、+補助記号	[、]	sp_S
0	×+名詞	[×]	b_B a_I ts_I u_E
0	Δ+記号	[Δ]	d_B e_I r_I u_I t_I a_E
0	Θ+記号	[Θ]	sh_B i:_I t_I a_E
0	Σ+記号	[Σ]	sh_B i_I g_I u_I m_I a_E
1	Φ+記号	[Φ]	f_B a_I i_E
0	Ψ+記号	[Ψ]	p_B u_I s_I a_I i_E
1	Ω+名詞	[Ω]	o:_B m_I u_E
